In [23]:
# !pip install git+https://github.com/dgunning/cord19.git

In [1]:
import pandas as pd
import pickle
import subprocess
import shutil
import hashlib
import nltk
import os

In [50]:
from cord import ResearchPapers
import featuretools as ft
from featuretools.nlp_primitives import UniversalSentenceEncoder


In [4]:
#path information
task='task1'# or 'task2' # specify task
root_path='/repo1/code/autoreview/'
data_path=root_path+'data/'+task+'/'
literature_path=data_path+'CORD-19-research-challenge/'#path to save retrieved articles abstract
sentence_file_name='hypercoagulable_sentences.tsv'
#sentence_embedding_file_name='hypercoagulable_sentence_USEfeatures.pkl'
keywords_of_interest =['anticoagulants','venous thromboembolism', 'thrombotic complications', 'hypercoagulability','clot formation', 'Thrombosis', 'Thrombotic', 'D-Dimer']


In [5]:
def download_literature():
    """
    Download literatures from CORD19, Google Scholar, and Bioarix.
    """
    #Download
    #Biorxiv
    subprocess.call(['curl -o collection.json https://connect.biorxiv.org/relate/collection_json.php?grp=181'], shell=True)
    #CORD19
    subprocess.call(['wget https://www.dropbox.com/s/osa58hx8rs5yl3t/metadata.csv?raw=1'],shell=True)
    #Google Scholar
    subprocess.call(['wget https://www.dropbox.com/s/5incr3c86sh43gq/hypercoagulable_fulltext.xlsx?raw=1'],shell=True)

    #Create directory to put literature files
    if not os.path.exists(literature_path):
        os.makedirs(literature_path)
    
    #move files to data_path
    shutil.move('./metadata.csv?raw=1',literature_path+'metadata_old.csv')
    shutil.move('./hypercoagulable_fulltext.xlsx?raw=1',literature_path+'hypercoagulable_fulltext.xlsx')
    shutil.move('./collection.json', literature_path+'biorxiv.json')


In [10]:
def merge2cord19_metadata():
    """
    Merge the three different dataframes into one dataframe following CORD19 structure
    """
    
    def hash(sourcedf,destinationdf,*column):
        columnName = ''
        destinationdf['sha'] = pd.DataFrame(sourcedf[list(column)].values.sum(axis=1))[0].str.encode('utf-8').apply(lambda x: (hashlib.sha512(x).hexdigest().upper()))
        destinationdf['cord_uid'] = destinationdf['sha'].apply(lambda x: x[0:8] )
    # hash(df,df,'ID','Salt')

    #Load the literature files as pd.Dataframe
    df_cord19_meta = pd.read_csv(literature_path+'metadata_old.csv')
    df_pap_gscholar = pd.read_excel(literature_path+'hypercoagulable_fulltext.xlsx')
    df_biomedRxiv = pd.json_normalize(pd.read_json(literature_path+'biorxiv.json')['rels'],'rel_authors',['rel_title', 'rel_doi',\
                                                                        'rel_link', 'rel_abs', \
                                                                        'rel_date', 'rel_site'])

    df_biomedRxiv = df_biomedRxiv.groupby(['rel_title', 'rel_doi', 'rel_link',
           'rel_abs', 'rel_date', 'rel_site'])['author_name'].agg(author_name=lambda x: ','.join(x)).reset_index()


    # Merge the three different dataframes into `df_cord19_meta_augmented`
    dic_pap2cord10 = {'Authors':'authors', 'Title': 'title', 
                      'Year':'publish_time',  'ArticleURL':'url', 'Source':'journal',
                       'DOI':'doi', 'Abstract':'abstract'}
    dic_biomedRxiv2cord10 = {
        'rel_title':'title', 'rel_doi':'doi' ,
        'rel_link':'url', 'rel_abs':'abstract', 
        'author_name':'authors', 'rel_date':'publish_time', 
        'rel_site':'journal'
    }

    #display(df_cord19_meta.head(1))

    df_pap_scholar_compatible = df_pap_gscholar[list(dic_pap2cord10.keys())]
    df_pap_scholar_compatible.columns = list(dic_pap2cord10.values())
    hash(df_pap_scholar_compatible,df_pap_scholar_compatible,'title')
    #display(df_pap_scholar_compatible.head(1))

    df_biomedRxiv_compatible = df_biomedRxiv[list(dic_biomedRxiv2cord10.keys())]
    df_biomedRxiv_compatible.columns = list(dic_biomedRxiv2cord10.values())
    hash(df_biomedRxiv_compatible,df_biomedRxiv_compatible,'title')
    #display(df_biomedRxiv_compatible.head(1))

    df_cord19_meta_augmented = df_cord19_meta.append(df_pap_scholar_compatible, ignore_index=True).append(df_biomedRxiv_compatible, ignore_index=True)
    df_cord19_meta_augmented['publish_time'] = df_cord19_meta_augmented['publish_time'].astype('datetime64')
    
    df_cord19_meta_augmented.to_csv(literature_path+'metadata.csv')

    #return df_cord19_meta_augmented



In [ ]:
def create_entityset(entityset_name, df):
    es = ft.EntitySet(entityset_name)
    u = df.abstract.str.split('.',expand=True).stack()

    df2 = pd.DataFrame({
        'ncord_uid': u.index.get_level_values(0) , 
        'sentence': u.values
    })

    es = es.entity_from_dataframe(entity_id="paper",
                                  dataframe=df,
                                  index = 'ncord_uid',
                                  make_index = True,
                                 )

    es = es.entity_from_dataframe(entity_id="sentence",
                                  dataframe=df2,
                                  index = 'sid',
                                  make_index = True,
                                 )

    es = es.add_relationship(ft.Relationship(es["paper"]["ncord_uid"],
                                       es["sentence"]["ncord_uid"]))

    return es


In [7]:
# Prepare merged `metadata` if not exist
download_literature()
merge2cord19_metadata()

In [25]:
# integrate with CORD19 package for screening/searching
research_papers = ResearchPapers.load(data_dir=data_path)

Loading metadata from /repo1/code/autoreview/data/task1/CORD-19-research-challenge
Cleaning metadata
Applying tags to metadata

Indexing research papers
Creating the BM25 index from the abstracts of the papers
Use index="text" if you want to index the texts of the paper instead
Finished Indexing in 93.0 seconds


In [42]:
# Search papers related to keywords
fields = ['cord_uid','sha','title','journal','authors','abstract','covid_related','virus','coronavirus','sars','published','when']
hypercoagulable =pd.DataFrame(columns = fields)

for item in keywords_of_interest:
    print('Searching term = ', item)
    temp  = research_papers.contains('treat',column='abstract').search(item,num_results=1000)
    hypercoagulable = hypercoagulable.append(temp.results[fields], ignore_index = True)

Searching term =  anticoagulants


W0615 17:38:54.650547 139790685652736 summarizer.py:432] Input text is expected to have at least 10 sentences.
W0615 17:38:54.652114 139790685652736 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W0615 17:38:54.672474 139790685652736 summarizer.py:432] Input text is expected to have at least 10 sentences.
W0615 17:38:54.673624 139790685652736 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W0615 17:38:54.674515 139790685652736 summarizer.py:370] Please add more sentences to the text. The number of reachable nodes is below 3
W0615 17:38:54.675325 139790685652736 summarizer.py:440] Couldn't get relevant sentences.
W0615 17:38:54.676416 139790685652736 summarizer.py:432] Input text is expected to have at least 10 sentences.
W0615 17:38:54.677319 139790685652736 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W0615 17:38:54.678159 139790685652736 summarizer.py:370] Please add more sentences to the text. The n

W0615 17:38:55.124567 139790685652736 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W0615 17:38:55.133930 139790685652736 summarizer.py:432] Input text is expected to have at least 10 sentences.
W0615 17:38:55.135462 139790685652736 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W0615 17:38:55.140389 139790685652736 summarizer.py:432] Input text is expected to have at least 10 sentences.
W0615 17:38:55.141722 139790685652736 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W0615 17:38:55.145667 139790685652736 summarizer.py:432] Input text is expected to have at least 10 sentences.
W0615 17:38:55.146917 139790685652736 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W0615 17:38:55.158102 139790685652736 summarizer.py:432] Input text is expected to have at least 10 sentences.
W0615 17:38:55.159608 139790685652736 summarizer.py:356] Input corpus is expected to have at least 10 do

W0615 17:38:55.578284 139790685652736 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W0615 17:38:55.598879 139790685652736 summarizer.py:432] Input text is expected to have at least 10 sentences.
W0615 17:38:55.599941 139790685652736 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W0615 17:38:55.606672 139790685652736 summarizer.py:432] Input text is expected to have at least 10 sentences.
W0615 17:38:55.607734 139790685652736 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W0615 17:38:55.611512 139790685652736 summarizer.py:432] Input text is expected to have at least 10 sentences.
W0615 17:38:55.612513 139790685652736 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W0615 17:38:55.621765 139790685652736 summarizer.py:432] Input text is expected to have at least 10 sentences.
W0615 17:38:55.622845 139790685652736 summarizer.py:356] Input corpus is expected to have at least 10 do

W0615 17:38:56.393789 139790685652736 summarizer.py:432] Input text is expected to have at least 10 sentences.
W0615 17:38:56.395182 139790685652736 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W0615 17:38:56.414462 139790685652736 summarizer.py:432] Input text is expected to have at least 10 sentences.
W0615 17:38:56.415755 139790685652736 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W0615 17:38:56.446877 139790685652736 summarizer.py:432] Input text is expected to have at least 10 sentences.
W0615 17:38:56.448179 139790685652736 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W0615 17:38:56.451604 139790685652736 summarizer.py:432] Input text is expected to have at least 10 sentences.
W0615 17:38:56.452539 139790685652736 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W0615 17:38:56.456778 139790685652736 summarizer.py:432] Input text is expected to have at least 10 sent

...

W0615 17:39:58.283850 139790685652736 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W0615 17:39:58.312914 139790685652736 summarizer.py:432] Input text is expected to have at least 10 sentences.
W0615 17:39:58.314383 139790685652736 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W0615 17:39:58.324486 139790685652736 summarizer.py:432] Input text is expected to have at least 10 sentences.
W0615 17:39:58.325753 139790685652736 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W0615 17:39:58.334722 139790685652736 summarizer.py:432] Input text is expected to have at least 10 sentences.
W0615 17:39:58.335948 139790685652736 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W0615 17:39:58.345190 139790685652736 summarizer.py:432] Input text is expected to have at least 10 sentences.
W0615 17:39:58.346303 139790685652736 summarizer.py:356] Input corpus is expected to have at least 10 do

W0615 17:39:58.917979 139790685652736 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W0615 17:39:58.925675 139790685652736 summarizer.py:432] Input text is expected to have at least 10 sentences.
W0615 17:39:58.926758 139790685652736 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W0615 17:39:58.931026 139790685652736 summarizer.py:432] Input text is expected to have at least 10 sentences.
W0615 17:39:58.931943 139790685652736 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W0615 17:39:58.946173 139790685652736 summarizer.py:432] Input text is expected to have at least 10 sentences.
W0615 17:39:58.947035 139790685652736 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W0615 17:39:58.971586 139790685652736 summarizer.py:432] Input text is expected to have at least 10 sentences.
W0615 17:39:58.972445 139790685652736 summarizer.py:356] Input corpus is expected to have at least 10 do

W0615 17:39:59.332161 139790685652736 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W0615 17:39:59.334694 139790685652736 summarizer.py:432] Input text is expected to have at least 10 sentences.
W0615 17:39:59.335458 139790685652736 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W0615 17:39:59.346119 139790685652736 summarizer.py:432] Input text is expected to have at least 10 sentences.
W0615 17:39:59.347066 139790685652736 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W0615 17:39:59.347754 139790685652736 summarizer.py:370] Please add more sentences to the text. The number of reachable nodes is below 3
W0615 17:39:59.348344 139790685652736 summarizer.py:440] Couldn't get relevant sentences.
W0615 17:39:59.349274 139790685652736 summarizer.py:432] Input text is expected to have at least 10 sentences.
W0615 17:39:59.349959 139790685652736 summarizer.py:356] Input corpus is expected to have at least 10

W0615 17:39:59.716348 139790685652736 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W0615 17:39:59.723371 139790685652736 summarizer.py:432] Input text is expected to have at least 10 sentences.
W0615 17:39:59.724316 139790685652736 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W0615 17:39:59.741823 139790685652736 summarizer.py:432] Input text is expected to have at least 10 sentences.
W0615 17:39:59.742934 139790685652736 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W0615 17:39:59.750494 139790685652736 summarizer.py:432] Input text is expected to have at least 10 sentences.
W0615 17:39:59.751537 139790685652736 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W0615 17:39:59.754728 139790685652736 summarizer.py:432] Input text is expected to have at least 10 sentences.
W0615 17:39:59.755845 139790685652736 summarizer.py:356] Input corpus is expected to have at least 10 do

W0615 17:40:00.141341 139790685652736 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W0615 17:40:00.154258 139790685652736 summarizer.py:432] Input text is expected to have at least 10 sentences.
W0615 17:40:00.155350 139790685652736 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W0615 17:40:00.185217 139790685652736 summarizer.py:432] Input text is expected to have at least 10 sentences.
W0615 17:40:00.186164 139790685652736 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W0615 17:40:00.206196 139790685652736 summarizer.py:432] Input text is expected to have at least 10 sentences.
W0615 17:40:00.207493 139790685652736 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W0615 17:40:00.223839 139790685652736 summarizer.py:432] Input text is expected to have at least 10 sentences.
W0615 17:40:00.225005 139790685652736 summarizer.py:356] Input corpus is expected to have at least 10 do

In [45]:
# Generate sentence level dataframe from all papers
es=create_entityset("covid19_complete", research_papers.metadata[fields])
fulldf = es['sentence'].df.merge(es['paper'].df[['ncord_uid','cord_uid','sha']])
fulldf.rename(columns={'sha':'paper_id'},inplace=True)
fulldf.to_csv(literature_path+'metadata_sentences_with_cord_uid.csv')


In [49]:
# Generate sentence level dataframe from hypercoagulabel papers
es_hyper=create_entityset("covid19", hypercoagulable[fields])
es_hyper['sentence'].df.to_csv(data_path+sentence_file_name, sep='\t')


In [51]:
# # Extract sentence embedding from UniversalSentenceEncoder
# It takes around 15 minutes
# fm_inv, features_inv = ft.dfs(entityset=es_hyper,
#                               target_entity='sentence',
#                               max_depth = 1,
#                               trans_primitives =[UniversalSentenceEncoder],
#                               verbose=True)
# fm_inv.filter(regex='UNIVERSAL_SENTENCE_ENCODER(sentence)*|ncord_uid',axis=1)
# fm_inv.to_pickle(data_path+'hypercoagulable_sentence_USEfeatures.pkl')

Built 10 features
Elapsed: 10:58 | Progress:  50%|████▉     


KeyboardInterrupt: 